# DS9

# Importing-important-libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

warnings.filterwarnings("ignore")

# loading dataset

In [2]:
df = pd.read_csv("Iris.csv")

In [3]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
df.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


# splitting data in training and testing set

In [6]:
train_set,test_set=train_test_split(df,train_size=0.70,shuffle=True)

In [7]:
X_train=train_set[["Id","SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]]
Y_train=train_set["Species"]
X_test=test_set[["Id","SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]]
Y_test = test_set["Species"]

# using naive bayes algorithm to test model and make predictions

In [8]:
#Initialize Gaussian Naive Bayes
clf = GaussianNB()

# Split-out validation dataset
array = df.values
X = array[:,1:5]
Y = array[:,5]

# One-third of data as a part of test set
validation_size = 0.33

seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Test options and evaluation metric
scoring = 'accuracy'

#Fitting the training set
clf.fit(X_train, Y_train) 

#Predicting for the Test Set
pred_clf = clf.predict(X_validation)

#Prediction Probability
prob_pos_clf = clf.predict_proba(X_validation)[:, 1]

#Create the prediction file by concatenation of the original data and predictions
#Reshaping needed to perform the concatenation
pred_clf_df = pd.DataFrame(pred_clf.reshape(50,1))
#Column renaming to indicate the predictions
pred_clf_df.rename(columns={0:'Prediction'}, inplace=True)

#reshaping the test dataset
X_validation_df = pd.DataFrame(X_validation.reshape(50,4))

#concatenating the two pandas datfrom sklearn.naive_bayes import GaussianNBaframes over the columns to create a prediction dataset
pred_outcome = pd.concat([X_validation_df, pred_clf_df], axis=1)

pred_outcome.rename(columns = {0:'SepalLengthCm', 1:'SepalWidthCm', 2:'PetalLengthCm', 3:'PetalWidthCm'}, inplace=True)

del df['Id']

#merging the prediction with original dataset
pred_comp = pd.merge(df,pred_outcome, on=['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm'])

#print top 10 lines of the final predictions
print((pred_comp).head(10))
print ("\n")

#Save the file to csv
pred_comp.to_csv('Predictions.csv', sep=',')

#Save the file to Excel
from pandas import ExcelWriter

writer = ExcelWriter('IrisPredictions.xlsx')
pred_comp.to_excel(writer,'Sheet1')
writer.save()


#Model Performance
#setting performance parameters
kfold = model_selection.KFold(n_splits=10, random_state=seed)

#calling the cross validation function
cv_results = model_selection.cross_val_score(GaussianNB(), X_train, Y_train, cv=kfold, scoring=scoring)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('NB accuracy', cv_results.mean(), cv_results.std())
print(msg)

  SepalLengthCm SepalWidthCm PetalLengthCm PetalWidthCm      Species  \
0           5.4          3.9           1.7          0.4  Iris-setosa   
1           4.9          3.1           1.5          0.1  Iris-setosa   
2           4.9          3.1           1.5          0.1  Iris-setosa   
3           4.9          3.1           1.5          0.1  Iris-setosa   
4           4.8          3.4           1.6          0.2  Iris-setosa   
5           5.1          3.5           1.4          0.3  Iris-setosa   
6           4.6          3.6             1          0.2  Iris-setosa   
7           5.2          3.4           1.4          0.2  Iris-setosa   
8           4.7          3.2           1.6          0.2  Iris-setosa   
9           5.2          4.1           1.5          0.1  Iris-setosa   

    Prediction  
0  Iris-setosa  
1  Iris-setosa  
2  Iris-setosa  
3  Iris-setosa  
4  Iris-setosa  
5  Iris-setosa  
6  Iris-setosa  
7  Iris-setosa  
8  Iris-setosa  
9  Iris-setosa  


NB accuracy: 0.970

# DS3: how to deal with missing values


we could find missing/corrupted data in a dataset and either drop those rows or columns, or decide to replace them with another value.
In Pandas, there are two very useful methods: isnull() and dropna() that will help us find columns of data with missing or corrupted data and drop those values. If we want to fill the invalid values with a placeholder value (for example, 0), you could use the fillna() method.
or we can do perform techniques:
Build another predictive model to predict the missing values – This could be a whole project in itself, so simple techniques are usually used here.
Use a model that can incorporate missing data – Like a random forest, or any tree-based method.

# As asked to do i have done two problems out of which one tests my practical knowledge and other tests my theoretical knowledge 